In [307]:
import io
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sb
from timeit import default_timer as timer
from tweepy import OAuthHandler
import xml.etree.ElementTree

# The Goal

Wrangle WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. The Twitter archive is great, but it only contains very basic tweet information. Additional gathering, then assessing and cleaning is required for better analyses and visualizations.

# Table of contents
1. [Gathering Data](#Gathering)
    1. [WeRateDogs Twitter archive](#tweets)
    2. [Image Predictions](#predictions)
    3. [Tweets Info](#tweets_info)
2. [Assessing Data](#Assessing)
    1. [Tweets Info assessing](#info_assessing)
    2. [WeRateDogs Twitter archive assessing](#tweets_assessing)
    3. [Image Predictions assessing](#predictions_assessing)
    4. [Issues Summary](#Issues_summary)
3. [Cleaning Data](#cleaning_data)
4. [Storing The Data](#storing)
5. [insights](#insights)
    1. [insight 1](#info_assessing)
    2. [insight 2](#tweets_assessing)
    3. [insight 3](#predictions_assessing)

# Gathering Data <a name="Gathering"></a>

I will import 3 datasets into pandas dataframes:

1. WeRateDogs Twitter archive - by importing csv from the system
2. tweet image predictions - tcv file using requests
3. tetweet info - using tweepy and twitter's api


### WeRateDogs Twitter archive - from pc  <a name="tweets"></a>

In [41]:
# read simple csv file on pc
dogs_tweets = pd.read_csv('twitter-archive-enhanced.csv')

# validate reading
print(dogs_tweets.shape)
dogs_tweets.head(2)

(2356, 17)


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


### Tweets Image Predictions - from url Image <a name="predictions"></a>

In [42]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'


def request_to_df(url, sep='\t', header=0):
    '''
    desc:
    ---------
    request csv file and import it into df
    
    input:
    ----------
    * url - link to csv file
    * sep - a string, Delimiter to use, a pandas parameter. /t for tab 
    * header - Row number(s) to use as the column names. row 0 is default
    
    output:
    -----------
    Return a dataframe named df
    '''
    
    urlData = requests.get(url).content
    df = pd.read_csv(io.StringIO(urlData.decode('utf-8')), sep=sep, header=header)
    return df


predictions = request_to_df(url)

# validate reading
print(predictions.shape)
predictions.head(2)

(2075, 12)


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


### tweet info - from api to df <a name="tweets_info"></a>

In [ ]:
# Code from Udacity
# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions

df_1 = dogs_tweets.copy()

consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
# df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
# change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv
# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API

tweet_ids = df_1.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
# print(end - start)
# print(fails_dict)

1: 892420643555336193
Fail
2: 892177421306343426
Fail
3: 891815181378084864
Fail
4: 891689557279858688
Fail
5: 891327558926688256
Fail
6: 891087950875897856
Fail
7: 890971913173991426
Fail
8: 890729181411237888
Fail
9: 890609185150312448
Fail
10: 890240255349198849
Fail
11: 890006608113172480
Fail
12: 889880896479866881
Fail
13: 889665388333682689
Fail
14: 889638837579907072
Fail
15: 889531135344209921
Fail
16: 889278841981685760
Fail
17: 888917238123831296
Fail
18: 888804989199671297
Fail
19: 888554962724278272
Fail
20: 888202515573088257
Fail
21: 888078434458587136
Fail
22: 887705289381826560
Fail
23: 887517139158093824
Fail
24: 887473957103951883
Fail
25: 887343217045368832
Fail
26: 887101392804085760
Fail
27: 886983233522544640
Fail
28: 886736880519319552
Fail
29: 886680336477933568
Fail
30: 886366144734445568
Fail
31: 886267009285017600
Fail
32: 886258384151887873
Fail
33: 886054160059072513
Fail
34: 885984800019947520
Fail
35: 885528943205470208
Fail
36: 885518971528720385
Fail
3

In [43]:
def json_to_df(txt_file):
    '''
    desc:
    ---------
    Convert json data from twiiter to into dataframe.
    The columns are hard coded for now
    
    input:
    ----------
    txt_file - a path to text file with json data from api
    
    output:
    -----------
    Return a dataframe named df
    '''
    
    tweet_info = []
    
    # read file into list
    with open(txt_file, "r") as f:
        for line in f:
            tweet = json.loads(line)
            tweet_info.append({
                "tweet_id": tweet["id"],
                "retweet_count": tweet["retweet_count"],
                "favorite_count": tweet["favorite_count"],"retweeted": tweet["retweeted"]                
            })
            
    # convert list into df
    df = pd.DataFrame(tweet_info, 
                      columns = ["tweet_id", "retweet_count", "favorite_count"])
    return df

In [44]:
tweet_info = json_to_df("tweet-json.txt")

# validate reading
print(tweet_info.shape)
tweet_info.head(5)

(2354, 3)


,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


# Assessing Data <a name="Assessing"></a>

I will assess the data from the 3 data frames visually and programmatically for quality and tidiness issues.
I will docuement each issue along the way and in the end of this part.

The data frame will be tested in following order:
1. tweets info
2. dogs tweets
3. predictions

## Tweets information <a name="info_assessing"></a>

In [45]:
display(tweet_info.head())
display(tweet_info.tail())

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


,tweet_id,retweet_count,favorite_count
2349,666049248165822465,41,111
2350,666044226329800704,147,311
2351,666033412701032449,47,128
2352,666029285002620928,48,132
2353,666020888022790149,532,2535


In [46]:
tweet_info.info()
tweet_info.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   tweet_id        2354 non-null   int64
 1   retweet_count   2354 non-null   int64
 2   favorite_count  2354 non-null   int64
dtypes: int64(3)
memory usage: 55.3 KB


,tweet_id,retweet_count,favorite_count
count,2.354000e+03,2354.000000,2354.000000
mean,7.426978e+17,3164.797366,8080.968564
std,6.852812e+16,5284.770364,11814.771334
min,6.660209e+17,0.000000,0.000000
25%,6.783975e+17,624.500000,1415.000000
50%,7.194596e+17,1473.500000,3603.500000
75%,7.993058e+17,3652.000000,10122.250000
max,8.924206e+17,79515.000000,132810.000000


In [47]:
tweet_info[tweet_info["tweet_id"].duplicated()]

,tweet_id,retweet_count,favorite_count


1. There are no any duplicated values, weird max / min values 
2. There isn't any null at all. 
3. The tweet_id column need to be convert to string.

## dogs tweets <a name="tweets_assessing"></a>

In [48]:
dogs_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

1. tweet_id need to be converted to str, because it is a key.
2. timestamp can be converted into date data type.
3. most of the fields retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, in_reply_to_status_id and in_reply_to_user_id have missing values. They maybe can be removed. If I will keep them, they need to str due to them id field, and the timestamp need to convert into timestamp.

In [49]:
# print all rows for trying to assest visually

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dogs_tweets)

                tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0     892420643555336193                    NaN                  NaN   
1     892177421306343426                    NaN                  NaN   
2     891815181378084864                    NaN                  NaN   
3     891689557279858688                    NaN                  NaN   
4     891327558926688256                    NaN                  NaN   
5     891087950875897856                    NaN                  NaN   
6     890971913173991426                    NaN                  NaN   
7     890729181411237888                    NaN                  NaN   
8     890609185150312448                    NaN                  NaN   
9     890240255349198849                    NaN                  NaN   
10    890006608113172480                    NaN                  NaN   
11    889880896479866881                    NaN                  NaN   
12    889665388333682689                    NaN                 

1. The source - can probaly remove the link and the html tags, just need to make sure there is different sources.
2. Some dogs don't have names or have weird name, like 'a' or 'the'. I will remove the names a, none and the.
3. not all dogs have type.

The first issue is a tidy issue - it seems the columns doggo, floofer, pupper and puppo can be one field. each include two possibiltes, the field name or None.

In [50]:
# Does the dog type field are binary? 

print(dogs_tweets.puppo.unique())
print(dogs_tweets.pupper.unique())
print(dogs_tweets.doggo.unique())
print(dogs_tweets.floofer.unique())

['None' 'puppo']
['None' 'pupper']
['None' 'doggo']
['None' 'floofer']


In [51]:
# Are There more then 1 type of source?

dogs_tweets.source.unique()

array(['<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'],
      dtype=object)

In [52]:
# check for weird dog's names

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dogs_tweets.name.value_counts())

None              745
a                  55
Charlie            12
Lucy               11
Oliver             11
Cooper             11
Penny              10
Tucker             10
Lola               10
Winston             9
Bo                  9
the                 8
Sadie               8
Daisy               7
Toby                7
Buddy               7
Bailey              7
an                  7
Leo                 6
Milo                6
Jack                6
Rusty               6
Scout               6
Jax                 6
Bella               6
Oscar               6
Dave                6
Stanley             6
Koda                6
Phil                5
Gus                 5
Sunny               5
Alfie               5
Chester             5
Louis               5
Bentley             5
Oakley              5
very                5
George              5
Sammy               5
Finn                5
Larry               5
Maximus             4
Walter              4
Boomer              4
Bruce     

In [21]:
# How many dogs with 1 letter name?

dogs_tweets[dogs_tweets['name'].apply(len)==1][['tweet_id','name']].groupby('name').count()

,tweet_id
name,
O,1
a,55


In [22]:
# How many dogs with 2 letter name?

dogs_tweets[dogs_tweets['name'].apply(len)==2][['tweet_id','name']].groupby('name').count()

,tweet_id
name,
Al,1
Bo,9
Ed,1
JD,1
Jo,1
Mo,1
an,7
by,1
my,1


In [24]:
# How many dogs with 3 letter name?

dogs_tweets[dogs_tweets['name'].apply(len)==3][['tweet_id','name']].groupby('name').count()

,tweet_id
name,
Ace,1
Aja,1
Alf,1
Amy,1
Ash,2
Ava,2
Ben,1
Blu,1
Bob,2


In [33]:
# find all non capitlize / all letters are lovwer case

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(dogs_tweets[dogs_tweets.name.str.islower()==True][['tweet_id', 'name']])

,tweet_id,name
22,887517139158093824,such
56,881536004380872706,a
118,869988702071779329,quite
169,859196978902773760,quite
193,855459453768019968,quite
335,832645525019123713,not
369,828650029636317184,one
542,806219024703037440,incredibly
649,792913359805018113,a
682,788552643979468800,mad


In [35]:
# How many dogs do not have real name?

dogs_tweets[dogs_tweets.name.str.islower()==True][['name']].count()

name    109
dtype: int64

Werid / none names all be remove so we all be able to analyze dogs names better. It seems the easist way is to remove all non capitlize names based if all is lower case

names example: 
not, all, my, by, old, an, a, O, None, the, very, this, his, such, quite, incredibly, mad, getting, actually, infuriatingm officially  

In [13]:
# understaning the the ratings columns 
dogs_tweets[['rating_numerator', 'rating_denominator']].describe()

,rating_numerator,rating_denominator
count,2356.000000,2356.000000
mean,13.126486,10.455433
std,45.876648,6.745237
min,0.000000,0.000000
25%,10.000000,10.000000
50%,11.000000,10.000000
75%,12.000000,10.000000
max,1776.000000,170.000000


1. The avg numerator is bigger then denominator which is correct.
2. There are dogs with 0 numerator and/or 0 denominator, which need checking.
3. max denominator of 170 - need testing

In [39]:
# find rating rating numerator == 0
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'max_colwidth', 200):
    display(dogs_tweets.query("rating_numerator == 0")[['tweet_id','text', 'expanded_urls', 'rating_numerator', 'rating_denominator']])

,tweet_id,text,expanded_urls,rating_numerator,rating_denominator
315,835152434251116546,When you're so blinded by your systematic plagiarism that you forget what day it is. 0/10 https://t.co/YbEJPkg4Ag,"https://twitter.com/dog_rates/status/835152434251116546/photo/1,https://twitter.com/dog_rates/status/835152434251116546/photo/1,https://twitter.com/dog_rates/status/835152434251116546/photo/1",0,10
1016,746906459439529985,"PUPDATE: can't see any. Even if I could, I couldn't reach them to pet. 0/10 much disappointment https://t.co/c7WXaB2nqX",https://twitter.com/dog_rates/status/746906459439529985/photo/1,0,10


In [55]:
# find rating denominator == 0
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'max_colwidth', 200):
    display(dogs_tweets.query("rating_denominator == 0")[['tweet_id', 'expanded_urls', 'text', 'rating_numerator', 'rating_denominator']])

,tweet_id,expanded_urls,text,rating_numerator,rating_denominator
313,835246439529840640,NaN,"@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",960,0


1. Ids 835152434251116546 and 746906459439529985 can be removed due to them not dog tweet.
2. Id 835246439529840640 seem more as a joke then real rating - will be removed

In [36]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'max_colwidth', 200):
    display(dogs_tweets[dogs_tweets.rating_denominator > 10][['text', 'expanded_urls', 'rating_numerator', 'rating_denominator']])

,text,expanded_urls,rating_numerator,rating_denominator
342,@docmisterio account started on 11/15/15,NaN,11,15
433,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,"https://twitter.com/dog_rates/status/820690176645140481/photo/1,https://twitter.com/dog_rates/status/820690176645140481/photo/1,https://twitter.com/dog_rates/status/820690176645140481/photo/1",84,70
784,"RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…","https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,http...",9,11
902,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,https://twitter.com/dog_rates/status/758467244762497024/video/1,165,150
1068,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ","https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,http...",9,11
1120,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,https://twitter.com/dog_rates/status/731156023742988288/photo/1,204,170
1165,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,https://twitter.com/dog_rates/status/722974582966214656/photo/1,4,20
1202,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,https://twitter.com/dog_rates/status/716439118184652801/photo/1,50,50
1228,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,https://twitter.com/dog_rates/status/713900603437621249/photo/1,99,90
1254,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,https://twitter.com/dog_rates/status/710658690886586372/photo/1,80,80


In [54]:
# 342 and 1663 can be removed - not dogs?
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'max_colwidth', 200):
    display(dogs_tweets.iloc[[342, 1663]][['text', 'expanded_urls', 'rating_numerator', 'rating_denominator', 'name']])

,text,expanded_urls,rating_numerator,rating_denominator,name
342,@docmisterio account started on 11/15/15,NaN,11,15,None
1663,"I'm aware that I could've said 20/16, but here at WeRateDogs we are very professional. An inconsistent rating scale is simply irresponsible",NaN,20,16,None


In [37]:
# 1068 and 784 the same dog? maybe all 'RT @dog_rates' need to be removed
 with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'max_colwidth', 200):
    display(dogs_tweets.iloc[[784, 1068]][['text', 'expanded_urls', 'rating_numerator', 'rating_denominator', 'name']])

,text,expanded_urls,rating_numerator,rating_denominator,name
784,"RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…","https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,http...",9,11,None
1068,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ","https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,http...",9,11,None


In [46]:
# check for numerator smaller then denominator

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'max_colwidth', 200):
    display(dogs_tweets[dogs_tweets.rating_denominator > dogs_tweets.rating_numerator][
        ['text', 'rating_numerator', 'rating_denominator']])

,text,rating_numerator,rating_denominator
45,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",5,10
229,"This is Jerry. He's doing a distinguished tongue slip. Slightly patronizing tbh. You think you're better than us, Jerry? 6/10 hold me back https://t.co/DkOBbwulw1",6,10
315,When you're so blinded by your systematic plagiarism that you forget what day it is. 0/10 https://t.co/YbEJPkg4Ag,0,10
342,@docmisterio account started on 11/15/15,11,15
387,"I was going to do 007/10, but the joke wasn't worth the &lt;10 rating",7,10
462,RT @dog_rates: Meet Herschel. He's slightly bigger than ur average pupper. Looks lonely. Could probably ride 7/10 would totally pet https:/…,7,10
485,RT @dog_rates: Meet Beau &amp; Wilbur. Wilbur stole Beau's bed from him. Wilbur now has so much room for activities. 9/10 for both pups https:/…,9,10
599,RT @dog_rates: Here we see a rare pouched pupper. Ample storage space. Looks alert. Jumps at random. Kicked open that door. 8/10 https://t.…,8,10
605,RT @dog_rates: Not familiar with this breed. No tail (weird). Only 2 legs. Doesn't bark. Surprisingly quick. Shits eggs. 1/10 https://t.co/…,1,10
730,Who keeps sending in pictures without dogs in them? This needs to stop. 5/10 for the mediocre road https://t.co/ELqelxWMrC,5,10


Because we can see ratings who are float let's check how much it happend. Anyway it will need fixing.

In [53]:
# check for float ratings

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'max_colwidth', 200):
    display(dogs_tweets[dogs_tweets['text'].str.contains(r"(\d+\.\d*\/\d+)")][
        ['text', 'rating_numerator', 'rating_denominator']])

C:\Users\user\anaconda3\envs\Udacity\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,text,rating_numerator,rating_denominator
45,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",5,10
340,"RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…",75,10
695,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",75,10
763,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,27,10
1689,I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,5,10
1712,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,26,10


## Dogs Type Predections <a name="predictions_assessing"></a>

In [56]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [59]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'max_colwidth', 200):
    display(predictions)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,1.566650e-01,True,Shetland_sheepdog,6.142850e-02,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,7.419170e-02,True,Rhodesian_ridgeback,7.201000e-02,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,1.385840e-01,True,bloodhound,1.161970e-01,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,3.606870e-01,True,miniature_pinscher,2.227520e-01,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,2.436820e-01,True,Doberman,1.546290e-01,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,2.637880e-01,True,Greater_Swiss_Mountain_dog,1.619920e-02,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,4.588540e-02,False,terrapin,1.788530e-02,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,5.827940e-02,True,fur_coat,5.444860e-02,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,1.459380e-02,False,golden_retriever,7.958960e-03,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,1.923050e-01,True,soft-coated_wheaten_terrier,8.208610e-02,True


In [65]:
# check duplicates

display(predictions['tweet_id'].duplicated().sum())
display(predictions['jpg_url'].duplicated().sum())

0

66

In [70]:
# Lets look at all the duplicates

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'max_colwidth', 200):
    display(predictions[predictions['jpg_url'].duplicated()])

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1297,752309394570878976,https://pbs.twimg.com/ext_tw_video_thumb/675354114423808004/pu/img/qL1R_nGLqa6lmkOx.jpg,1,upright,0.303415,False,golden_retriever,0.181351,True,Brittany_spaniel,0.162084,True
1315,754874841593970688,https://pbs.twimg.com/media/CWza7kpWcAAdYLc.jpg,1,pug,0.272205,True,bull_mastiff,0.251530,True,bath_towel,0.116806,False
1333,757729163776290825,https://pbs.twimg.com/media/CWyD2HGUYAQ1Xa7.jpg,2,cash_machine,0.802333,False,schipperke,0.045519,True,German_shepherd,0.023353,True
1345,759159934323924993,https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg,1,Irish_terrier,0.254856,True,briard,0.227716,True,soft-coated_wheaten_terrier,0.223263,True
1349,759566828574212096,https://pbs.twimg.com/media/CkNjahBXAAQ2kWo.jpg,1,Labrador_retriever,0.967397,True,golden_retriever,0.016641,True,ice_bear,0.014858,False
1364,761371037149827077,https://pbs.twimg.com/tweet_video_thumb/CeBym7oXEAEWbEg.jpg,1,brown_bear,0.713293,False,Indian_elephant,0.172844,False,water_buffalo,0.038902,False
1368,761750502866649088,https://pbs.twimg.com/media/CYLDikFWEAAIy1y.jpg,1,golden_retriever,0.586937,True,Labrador_retriever,0.398260,True,kuvasz,0.005410,True
1387,766078092750233600,https://pbs.twimg.com/media/ChK1tdBWwAQ1flD.jpg,1,toy_poodle,0.420463,True,miniature_poodle,0.132640,True,Chesapeake_Bay_retriever,0.121523,True
1407,770093767776997377,https://pbs.twimg.com/media/CkjMx99UoAM2B1a.jpg,1,golden_retriever,0.843799,True,Labrador_retriever,0.052956,True,kelpie,0.035711,True
1417,771171053431250945,https://pbs.twimg.com/media/CVgdFjNWEAAxmbq.jpg,3,Samoyed,0.978833,True,Pomeranian,0.012763,True,Eskimo_dog,0.001853,True


In [71]:
# One duplicate for example

url='https://pbs.twimg.com/media/CWza7kpWcAAdYLc.jpg'
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'max_colwidth', 200):
    display(predictions[predictions['jpg_url']==url])

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
591,679158373988876288,https://pbs.twimg.com/media/CWza7kpWcAAdYLc.jpg,1,pug,0.272205,True,bull_mastiff,0.25153,True,bath_towel,0.116806,False
1315,754874841593970688,https://pbs.twimg.com/media/CWza7kpWcAAdYLc.jpg,1,pug,0.272205,True,bull_mastiff,0.25153,True,bath_towel,0.116806,False


In [66]:
# check for wierd numbers, like for img_num

predictions.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


### Issues summary: 
<a name="Issues_summary"></a>

* Tidy issues
    1. **for all datasets** convert id column to str 
    2. **Dog tweets:** timestamp to date
    3. **Dog tweets:** dog types need to change to one field (using pivot table)
* Quality issues
    1. **Dog tweets:** source field remove html tags
    2. **Dog tweets:** Remove weird  and not names names (using all lower)
    3. **Dog tweets:** dog type None need to be convert to NaN
    4. **Dog tweets:** Remove non dogs ratings = > 835152434251116546, 746906459439529985, 835246439529840640
    5. **Dog tweets:** Fix incorrect ratings
    6. **Dog tweets:** remove retweets as they can be the same dog
    7. **Dog tweets:** Extended url name need to be change to tweet_url, so it will be more clear
    8. **Predictions:** Remove duplicate urls
    9. **Predictions:** change for better name, like prediction_jpg_url

# Cleaning Data 
<a name="cleaning_data"></a>

At this part I will clean the datasets, based on the resulst I got in the former part. The dataframes will first copied and then will be cleaned.
The format of this part is: 

**Define**

What the issue?

**Code**

The code itself

**Test**

Testing the code did his job

In [217]:
# Copy the datasets

tweet_info_clean = tweet_info.copy()
dogs_tweets_clean = dogs_tweets.copy()
predictions_clean = predictions.copy()

# test the copy work

display(tweet_info_clean.head(1))
display(dogs_tweets_clean.head(1))
display(predictions_clean.head(1))

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True


## Tidy Issues

**Define**

For all dataframes id field is int, it need to be converted to string, due to it not use for calculation.

**Code**

In [218]:
tweet_info_clean['tweet_id'] = tweet_info_clean['tweet_id'].astype(str)
dogs_tweets_clean['tweet_id'] = dogs_tweets_clean['tweet_id'].astype(str)
predictions_clean['tweet_id'] = predictions_clean['tweet_id'].astype(str)

**Test**

In [219]:
display(tweet_info_clean.info())
display(dogs_tweets_clean.info())
display(predictions_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2354 non-null   object
 1   retweet_count   2354 non-null   int64 
 2   favorite_count  2354 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 55.3+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   object 
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   object 
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


None

**Define**

For dogs tweets dataframe date fields is object, it need to be converted to date, so it will be more useful.

**Code**

In [220]:
# format 2017-08-01 16:23:56 +0000
dogs_tweets_clean['timestamp'] =  pd.to_datetime(dogs_tweets_clean['timestamp'])

**Test**

In [221]:
dogs_tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   object             
 1   in_reply_to_status_id       78 non-null     float64            
 2   in_reply_to_user_id         78 non-null     float64            
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source                      2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         181 non-null    float64            
 7   retweeted_status_user_id    181 non-null    float64            
 8   retweeted_status_timestamp  181 non-null    object             
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2356 non-null   int64           

**Define**

In the dog tweets dataset there are 4 fields to dog type. It will be better with one column, based on the tidy rules.
First None will be need to be replaced for NaN

**Code**

In [222]:
dogs_tweets_clean.replace('None', np.NaN, inplace=True)

# testing

print(dogs_tweets_clean.puppo.unique())
print(dogs_tweets_clean.pupper.unique())
print(dogs_tweets_clean.doggo.unique())
print(dogs_tweets_clean.floofer.unique())

[nan 'puppo']
[nan 'pupper']
[nan 'doggo']
[nan 'floofer']


In [223]:
# melt all dogs type to one field. 
# WARNING! to run this cell again, one need to run the copy function again!

dog_columns_values = dogs_tweets_clean.columns[:-4].tolist()
dog_columns_vars = dogs_tweets_clean.columns[-4:].tolist()

dogs_tweets_clean = pd.melt(dogs_tweets_clean, id_vars=dog_columns_values, value_vars=dog_columns_vars, var_name='dog_type')
dogs_tweets_clean.drop(columns=['dog_type'], inplace=True)

In [224]:
dogs_tweets_clean = dogs_tweets_clean.drop_duplicates()

**Test**

In [225]:
print(dogs_tweets_clean.shape)
print(dogs_tweets_clean.value.unique())
dogs_tweets_clean.head(1)

(2750, 14)
[nan 'doggo' 'floofer' 'pupper' 'puppo']


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,value
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,NaN


Probably the are dogs with more then one type

In [226]:
duplicate_dogs_types = dogs_tweets_clean[dogs_tweets_clean.tweet_id.duplicated()][['tweet_id', 'value']]
duplicate_dogs_types

,tweet_id,value
2365,890240255349198849,NaN
2399,884162670584377345,NaN
2402,883360690899218434,floofer
2455,872967104147763200,NaN
2464,871515927908634625,NaN
...,...,...
8015,752519690950500352,puppo
8029,751132876104687617,puppo
8103,744995568523612160,puppo
8116,743253157753532416,puppo


In [227]:
# lets test one case
dogs_tweets_clean[dogs_tweets_clean.tweet_id == '890240255349198849']

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,value
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo
2365,890240255349198849,NaN,NaN,2017-07-26 15:59:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,NaN


In [228]:
# How many dogs of each type?

duplicate_dogs_types.value.value_counts()

pupper     257
puppo       30
floofer     10
Name: value, dtype: int64

In [229]:
# There are duplicated when droping only NaN?

duplicate_dogs_types.dropna().duplicated().sum()

0

In [230]:
# drop all NaN duplicated dogs

drop_duplicates_arr = duplicate_dogs_types[duplicate_dogs_types['value'].isna()].index
dogs_tweets_clean = dogs_tweets_clean.drop(drop_duplicates_arr)
dogs_tweets_clean.duplicated().sum()

0

In [231]:
dogs_tweets_clean.shape

(2653, 14)

In [254]:
dogs_tweets_clean.rename(columns={'value':'dog_type'}, inplace=True)

**Define**

In the dog tweets dataset the source field include html tags, there are not relevent for our analsys

**Code**

In [248]:
def remove_tags(text):
    '''
    desc:
    ---------
    Clean html tags from string
    
    input:
    ----------
    * text - a string (with html tags)
    
    output:
    -----------
    Return a string without html tags
    '''
    
    return ''.join(xml.etree.ElementTree.fromstring(text).itertext())

In [247]:
dogs_tweets_clean['source'] = dogs_tweets_clean.source.apply(remove_tags)

**Test**

In [249]:
dogs_tweets_clean.source.unique()

array(['Twitter for iPhone', 'Twitter Web Client', 'Vine - Make a Scene',
       'TweetDeck'], dtype=object)

**Define**

In the dog tweets dataset there are names who do not look like dogs names

**Code**

In [261]:
not_dog_name_index = dogs_tweets_clean[dogs_tweets_clean.name.str.islower()==True].index
dogs_tweets_clean = dogs_tweets_clean.drop(not_dog_name_index)

0

**Test**

In [262]:
dogs_tweets_clean[dogs_tweets_clean.name.str.islower()==True].name.count()

0

**Define**

In the dog tweets dataset there are few post with ratings which are not really dogs ratings

* *835152434251116546*

* *746906459439529985*

* *835246439529840640*

**Code**

In [270]:
jokes_not_dogs_arr = dogs_tweets_clean.query(
    "tweet_id in ('835246439529840640', '746906459439529985', '835152434251116546')").index
dogs_tweets_clean = dogs_tweets_clean.drop(jokes_not_dogs_arr)

**Test**

In [274]:
dogs_tweets_clean.query("tweet_id in ('835246439529840640', '746906459439529985', '835152434251116546')").tweet_id.count()

0

**Define**

For better reading the following column names need to be change:

1.Dog tweets: Extended url name need to be change to tweets_url

2.Predictions: jpg_url change for clear name, like prediction_img_url 

**Code**

In [276]:
dogs_tweets_clean.rename(columns={'expanded_urls':'tweets_url'}, inplace=True)
predictions_clean.rename(columns={'jpg_url':'prediction_img_url'}, inplace=True)

**Test**

In [277]:
print(dogs_tweets_clean.info())
print(predictions_clean.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2527 entries, 0 to 8151
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2527 non-null   object             
 1   in_reply_to_status_id       81 non-null     float64            
 2   in_reply_to_user_id         81 non-null     float64            
 3   timestamp                   2527 non-null   datetime64[ns, UTC]
 4   source                      2527 non-null   object             
 5   text                        2527 non-null   object             
 6   retweeted_status_id         202 non-null    float64            
 7   retweeted_status_user_id    202 non-null    float64            
 8   retweeted_status_timestamp  202 non-null    object             
 9   tweets_url                  2466 non-null   object             
 10  rating_numerator            2527 non-null   int64           

**Define**

Dog tweets: remove retweets as they can be the same dog. There are 202 retweets.

**Code**

In [286]:
dogs_tweets_clean = dogs_tweets_clean[dogs_tweets_clean.retweeted_status_id.isna()]

**Test**

In [289]:
dogs_tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2325 entries, 0 to 8151
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   tweet_id                  2325 non-null   object             
 1   in_reply_to_status_id     81 non-null     float64            
 2   in_reply_to_user_id       81 non-null     float64            
 3   timestamp                 2325 non-null   datetime64[ns, UTC]
 4   source                    2325 non-null   object             
 5   text                      2325 non-null   object             
 6   retweeted_status_user_id  0 non-null      float64            
 7   tweets_url                2265 non-null   object             
 8   rating_numerator          2325 non-null   int64              
 9   rating_denominator        2325 non-null   int64              
 10  name                      1537 non-null   object             
 11  dog_type         

In [288]:
col_to_drop = ['retweeted_status_timestamp', 'retweeted_status_id', 'retweeted_status_timestamp', 'retweeted_status_user_id']
dogs_tweets_clean.drop(columns=col_to_drop, inplace=True)

C:\Users\user\anaconda3\envs\Udacity\lib\site-packages\pandas\core\frame.py:4160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


**Define**

Predictions: Remove duplicate urls

**Code**

In [292]:
predictions_clean.drop_duplicates(subset=['prediction_img_url'], inplace=True)

**Test**

In [293]:
predictions_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2009 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            2009 non-null   object 
 1   prediction_img_url  2009 non-null   object 
 2   img_num             2009 non-null   int64  
 3   p1                  2009 non-null   object 
 4   p1_conf             2009 non-null   float64
 5   p1_dog              2009 non-null   bool   
 6   p2                  2009 non-null   object 
 7   p2_conf             2009 non-null   float64
 8   p2_dog              2009 non-null   bool   
 9   p3                  2009 non-null   object 
 10  p3_conf             2009 non-null   float64
 11  p3_dog              2009 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 162.8+ KB


In [295]:
predictions_clean['prediction_img_url'].duplicated().sum()

0

**Define**

In dog tweets there are more then 1 rows with incorrect values, where int need to be float

**Code**

In [297]:
# first lets convert to float from int

dogs_tweets_clean['rating_numerator'] = dogs_tweets_clean['rating_numerator'].astype(float)
dogs_tweets_clean['rating_denominator'] = dogs_tweets_clean['rating_denominator'].astype(float)

<ipython-input-297-3ba80e920bfd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dogs_tweets_clean['rating_numerator'] = dogs_tweets_clean['rating_numerator'].astype(float)
<ipython-input-297-3ba80e920bfd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dogs_tweets_clean['rating_denominator'] = dogs_tweets_clean['rating_denominator'].astype(float)


In [300]:
# check for float ratings

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'max_colwidth', 200):
    display(dogs_tweets_clean[dogs_tweets_clean['text'].str.contains(r"(\d+\.\d*\/\d+)")][
        ['text', 'rating_numerator', 'rating_denominator']])

C:\Users\user\anaconda3\envs\Udacity\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,text,rating_numerator,rating_denominator
45,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",5.0,10.0
695,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",75.0,10.0
763,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,27.0,10.0
1689,I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,5.0,10.0
1712,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,26.0,10.0
5475,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,27.0,10.0


In [301]:
dogs_tweets_clean.loc[45, 'rating_numerator'] = 13.5
dogs_tweets_clean.loc[695, 'rating_numerator'] = 9.75
dogs_tweets_clean.loc[763, 'rating_numerator'] = 11.27
dogs_tweets_clean.loc[1689, 'rating_numerator'] = 9.5
dogs_tweets_clean.loc[1712, 'rating_numerator'] = 11.26
dogs_tweets_clean.loc[5475, 'rating_numerator'] = 11.26

C:\Users\user\anaconda3\envs\Udacity\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


**Test**

In [302]:
# need to check one by one

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'max_colwidth', 200):
    display(dogs_tweets_clean[dogs_tweets_clean['text'].str.contains(r"(\d+\.\d*\/\d+)")][
        ['text', 'rating_numerator', 'rating_denominator']])

C:\Users\user\anaconda3\envs\Udacity\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,text,rating_numerator,rating_denominator
45,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",13.50,10.0
695,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",9.75,10.0
763,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,11.27,10.0
1689,I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,9.50,10.0
1712,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,11.26,10.0
5475,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,11.26,10.0


# Create New Dataset
<a name="storing"></a>

In [303]:
tweets_prediction_merge = dogs_tweets_clean.merge(predictions_clean, left_on='tweet_id', right_on='tweet_id')
twitter_archive_master = tweets_prediction_merge.merge(tweet_info_clean, left_on='tweet_id', right_on='tweet_id')
twitter_archive_master.shape

(2127, 25)

In [304]:
twitter_archive_master.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_user_id,tweets_url,rating_numerator,rating_denominator,...,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56+00:00,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,NaN,https://twitter.com/dog_rates/status/892420643...,13.0,10.0,...,0.097049,False,bagel,0.085851,False,banana,0.07611,False,8853,39467


In [305]:
twitter_archive_master.to_csv('twitter_archive_master.csv', index=False)

# Insights 
<a name="insights"></a>

## Insight 1
<a name="insight1"></a>

## Insight 2
<a name="insight2"></a>

## Insight 3 
<a name="insight3"></a>